In [1]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ee.Authenticate()

Enter verification code: 4/1AeaYSHBfmAun-1tTYsK2imqw-ljdZqZJnuf1NDoGiG30wg0O8SgmKy4DfR4

Successfully saved authorization token.


In [3]:
ee.Initialize()

# Shapefiles

In [4]:
Map=geemap.Map(center=[70, -40], zoom=10)

shp_path='/Users/raf/Documents/Columbia/Research/Albedo/Data/GrIS_polygon/GrIS_polygon.shp'
shp=geemap.shp_to_ee(shp_path)
geo=shp.geometry() 

Map.centerObject(shp, 3)
Map.addLayer(shp, {}, 'Greenland')

Map.addLayerControl()
Map

Map(center=[73.05915048517876, -41.91735811925366], controls=(WidgetControl(options=['position', 'transparent_…

# Data Management
Import datasets, select bands, then calculate Normalized Difference Vegetation Index (NDVI)

In [5]:
dataset=ee.ImageCollection("MODIS/006/MOD10A1")

# Get bands

In [6]:
def getbands(image):
    return image.select(['Snow_Albedo_Daily_Tile'])

In [7]:
bands = dataset.map(getbands)

# Filter Bounds

In [8]:
def clipgeo(image):
    return image.clip(geo)

In [9]:
bandsclip=bands.map(clipgeo)

# Filter Dates

In [10]:
years=range(2000,2023)

In [11]:
def yearly_image(year):
    start_date=ee.Date.fromYMD(year, 6, 1) 
    end_date=start_date.advance(92, "day")
    collection=bandsclip.filterDate(start_date, end_date) 
    return collection

In [12]:
collection={}

In [13]:
for year in years:
    collection[year]=yearly_image(year)

# Export Images

In [103]:
for yr in range(2000,2023):
    geemap.ee_export_image_collection_to_drive(collection[yr], folder='googleearthengine_MOD10A1_snow_albedo_daily_tile_2022', scale=500, region=geo, crs='EPSG:4326')

Total number of images: 92

